In [ ]:
# GARCH Financial Analysis: Interactive Exploration

This notebook provides an interactive way to explore the GARCH analysis results for NVIDIA and NASDAQ-100. It builds upon the comprehensive analysis pipeline and allows for custom exploration of volatility dynamics, spillover effects, and causal relationships.

## 🎯 Quick Start Guide
1. Run all cells sequentially to reproduce the analysis
2. Modify parameters to explore different scenarios  
3. Add your own analysis cells at the bottom

---


In [ ]:
# Setup and imports
import os
import sys
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Add src directory to Python path
sys.path.append('../src')

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

# Configuration
TICKERS = ['NVDA', '^NDX']
NAMES = {'NVDA': 'NVIDIA', '^NDX': 'NASDAQ-100'}
COLORS = {'NVDA': '#76B900', '^NDX': '#1f77b4'}

print("✅ Setup complete! Ready for GARCH analysis exploration.")


In [ ]:
# Load and display data overview
data = {}
for ticker in TICKERS:
    try:
        df = pd.read_csv(f'../data/{ticker}_cleaned.csv', index_col=0, parse_dates=True)
        data[ticker] = df
        print(f"✅ {ticker}: {len(df)} observations ({df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')})")
    except FileNotFoundError:
        print(f"❌ Data file not found for {ticker}")

# Show data structure
if data:
    print(f"\n📊 Data columns: {list(data[TICKERS[0]].columns)}")
    display(data[TICKERS[0]].head())


In [ ]:
# Interactive price and returns visualization
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Price levels
for ticker in TICKERS:
    if ticker in data:
        axes[0].plot(data[ticker].index, data[ticker]['Adj Close'], 
                    label=NAMES[ticker], color=COLORS[ticker], linewidth=2)

axes[0].set_title('Price Levels: NVIDIA vs NASDAQ-100 (2014-2023)', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Price ($)', fontsize=12)
axes[0].legend(fontsize=12)
axes[0].grid(True, alpha=0.3)

# Highlight COVID-19 period
axes[0].axvspan(pd.Timestamp('2020-02-01'), pd.Timestamp('2020-04-30'), 
               color='red', alpha=0.2, label='COVID-19 Crisis')

# Log returns
for ticker in TICKERS:
    if ticker in data:
        axes[1].plot(data[ticker].index, data[ticker]['Log Return'], 
                    label=NAMES[ticker], color=COLORS[ticker], linewidth=1.5, alpha=0.7)

axes[1].set_title('Daily Log Returns', fontsize=16, fontweight='bold')
axes[1].set_ylabel('Log Return', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].legend(fontsize=12)
axes[1].grid(True, alpha=0.3)

# Highlight COVID-19 period
axes[1].axvspan(pd.Timestamp('2020-02-01'), pd.Timestamp('2020-04-30'), 
               color='red', alpha=0.2)

plt.tight_layout()
plt.show()

# Calculate and display correlation
if len(data) >= 2:
    ret_nvda = data['NVDA']['Log Return'].dropna()
    ret_ndx = data['^NDX']['Log Return'].dropna()
    ret_nvda, ret_ndx = ret_nvda.align(ret_ndx, join='inner')
    correlation = ret_nvda.corr(ret_ndx)
    print(f"\n🔗 Overall correlation: {correlation:.4f} ({'Strong' if abs(correlation) > 0.7 else 'Moderate' if abs(correlation) > 0.5 else 'Weak'} positive correlation)")


In [ ]:
# Load and display GARCH analysis results
print("🏆 GARCH MODEL RESULTS")
print("=" * 40)

# Model comparison
try:
    garch_comparison = pd.read_csv('../figures/garch_model_comparison.csv')
    print("\n📊 Model Comparison (AIC/BIC):")
    key_cols = ['Ticker', 'Order', 'AIC', 'BIC', 'LogLik']
    if all(col in garch_comparison.columns for col in key_cols):
        display(garch_comparison[key_cols].round(4))
except FileNotFoundError:
    print("❌ GARCH comparison results not found")

# Volatility statistics
for ticker in TICKERS:
    try:
        vol_stats = pd.read_csv(f'../figures/volatility_stats_{ticker}.csv', index_col=0)
        print(f"\n📈 {NAMES[ticker]} Volatility Statistics:")
        display(vol_stats.round(4))
    except FileNotFoundError:
        print(f"❌ Volatility statistics not found for {ticker}")

# Granger causality results
try:
    granger_results = pd.read_csv('../figures/granger_causality_summary.csv', index_col=0)
    print("\n🔄 Granger Causality Results:")
    display(granger_results[['F-stat', 'p-value', 'Significant (5%)', 'Direction']].round(4))
except FileNotFoundError:
    print("❌ Granger causality results not found")

print("\n✅ All quantitative results loaded successfully!")


In [ ]:
## 🔬 Custom Analysis Section

**Add your own analysis cells below this point!**

### Ideas for exploration:
- Different rolling window sizes for correlation analysis
- Event studies around specific dates (earnings, Fed announcements)
- Comparison of volatility regimes
- Out-of-sample forecasting experiments
- Portfolio optimization using GARCH estimates

### Access to all results:
- **Data**: `../data/` - Raw and cleaned time series
- **Figures**: `../figures/` - All plots and statistical tables  
- **Results**: `../results/` - GARCH models and summary report
- **Source Code**: `../src/` - All analysis modules

---

**💡 Tip**: Use `python ../run_analysis.py` to regenerate all results if needed.
